# Modelagem de Pedidos

## Objetivo deste notebook

O objetivo deste notebook é realizar a **modelagem analítica do nível de pedido**, integrando informações dos datasets `orders` e `order_items`.

Nesta etapa, deixamos de analisar os datasets de forma isolada e passamos a **construir entidades analíticas**, onde cada linha representa um pedido com métricas agregadas e interpretáveis.

O foco não está em métricas avançadas de negócio, mas sim em:
- Construção correta de métricas básicas
- Clareza conceitual sobre o que cada métrica representa
- Preparação de uma base sólida para análises posteriores

## Datasets utilizados

Neste notebook, utilizaremos os seguintes datasets:

- `olist_orders_dataset.csv`: informações gerais sobre os pedidos (status, datas, cliente)
- `olist_order_items_dataset.csv`: informações detalhadas dos itens de cada pedido (preço, frete, quantidade de itens)

A junção desses datasets permitirá analisar os pedidos de forma mais completa, combinando **estrutura temporal** com **informações financeiras e operacionais**.

## Abordagem de modelagem

A modelagem seguirá uma abordagem incremental e explicativa:

1. Agregação dos dados de `order_items` no nível de pedido
2. Construção de métricas básicas por pedido
3. Junção do resultado com o dataset `orders`
4. Validação das métricas criadas
5. Análise exploratória da distribuição dessas métricas

Cada etapa será construída de forma consciente, evitando cálculos implícitos ou métricas pouco interpretáveis.

## Métricas que serão construídas

Durante este notebook, construiremos métricas básicas no nível de pedido, tais como:

- Quantidade de itens por pedido
- Valor total dos itens do pedido
- Valor total do frete
- Valor total do pedido (itens + frete)
- Ticket médio por pedido
- Relação entre número de itens e valor do pedido

Essas métricas serão sempre analisadas em conjunto com seu significado e limitações.

## Checklist da modelagem

Ao final deste notebook, esperamos ter clareza sobre:

- [ ] Como agregar corretamente itens no nível de pedido
- [ ] Como construir métricas básicas a partir de dados transacionais
- [ ] Distribuição das métricas por pedido
- [ ] Presença de assimetrias ou caudas longas
- [ ] Limitações analíticas da modelagem criada

## Observações importantes

Este notebook **não tem como objetivo**:

- Criar KPIs corporativos
- Avaliar performance financeira da empresa
- Otimizar estratégias de negócio

O foco está exclusivamente no **pensamento analítico**, na correta transformação dos dados e na interpretação responsável das métricas criadas.

## Carregamento dos datasets

Nesta etapa realizamos a leitura dos datasets `orders.csv` e `order_items.csv`.

O objetivo aqui é apenas carregar os dados necessários para a modelagem,
sem realizar qualquer tipo de transformação ou agregação.

In [131]:
import pandas as pd
import numpy as np

ORDERS_PATH = "../data/raw/olist_orders_dataset.csv"
ORDER_ITEMS_PATH = "../data/raw/olist_order_items_dataset.csv"

orders_df = pd.read_csv(ORDERS_PATH)
order_items_df = pd.read_csv(ORDER_ITEMS_PATH)

## Revisão semântica dos dados de itens de pedido

Antes de iniciar a agregação, é importante revisar o significado das colunas
presentes no dataset `order_items`, pois elas serão a base da modelagem.

Nesta etapa, buscamos responder:
- O que representa cada linha?
- Quais colunas podem ser agregadas?
- Quais colunas são apenas identificadores?

In [132]:
order_items_df.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [133]:
orders_df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


## Agregação dos itens no nível de pedido

O dataset `order_items` está no nível de item, ou seja, cada linha representa
um produto dentro de um pedido.

Para permitir análises no nível de pedido, é necessário agregar essas
informações, de forma que cada pedido passe a ser representado por uma única linha.

Nesta etapa, realizamos:
- A agregação dos itens por `order_id`
- A construção de métricas básicas no nível de pedido

In [134]:
pedido_agregado = (
    order_items_df
    .groupby('order_id')
    .agg(
        qtd_itens_pedido=('order_item_id', 'count'),
        valor_total_itens=('price', 'sum'),
        valor_total_frete=('freight_value', 'sum')
    )
    .reset_index()
)

In [135]:
pedido_agregado.head()

,order_id,qtd_itens_pedido,valor_total_itens,valor_total_frete
0,00010242fe8c5a6d1ba2dd792cb16214,1,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,199.90,18.14


## Construção de métricas derivadas

Após a agregação dos itens no nível de pedido, é possível construir métricas
derivadas a partir das informações já consolidadas.

Nesta etapa, criamos métricas que combinam valores agregados, permitindo
uma interpretação mais completa do comportamento dos pedidos.

In [136]:
pedido_agregado['valor_total_pedido'] = (
    pedido_agregado['valor_total_itens'] +
    pedido_agregado['valor_total_frete']
)

pedido_agregado['ticket_medio_item'] = (
    pedido_agregado['valor_total_itens'] /
    pedido_agregado['qtd_itens_pedido']
)

In [137]:
pedido_agregado[['qtd_itens_pedido',
                  'valor_total_itens',
                  'valor_total_frete',
                  'valor_total_pedido',
                  'ticket_medio_item']].head()

,qtd_itens_pedido,valor_total_itens,valor_total_frete,valor_total_pedido,ticket_medio_item
0,1,58.90,13.29,72.19,58.90
1,1,239.90,19.93,259.83,239.90
2,1,199.00,17.87,216.87,199.00
3,1,12.99,12.79,25.78,12.99
4,1,199.90,18.14,218.04,199.90


## Análise da distribuição das métricas por pedido

Nesta etapa, analisamos a distribuição das métricas construídas no nível de pedido,
com o objetivo de compreender:

- Concentração dos valores
- Presença de assimetrias ou caudas longas
- Relação entre média, mediana e quartis

Essa análise é fundamental para evitar interpretações incorretas e para entender
se métricas agregadas representam bem o comportamento típico dos pedidos.

In [138]:
pedido_agregado[
    ['qtd_itens_pedido',
     'valor_total_itens',
     'valor_total_pedido',
     'ticket_medio_item']
].describe()

,qtd_itens_pedido,valor_total_itens,valor_total_pedido,ticket_medio_item
count,98666.000000,98666.000000,98666.000000,98666.000000
mean,1.141731,137.754076,160.577638,125.919255
std,0.538452,210.645145,220.466087,190.985636
min,1.000000,0.850000,9.590000,0.850000
25%,1.000000,45.900000,61.980000,41.990000
50%,1.000000,86.900000,105.290000,79.000000
75%,1.000000,149.900000,176.870000,139.900000
max,21.000000,13440.000000,13664.080000,6735.000000


In [139]:
dist_itens = (
    pedido_agregado
    .groupby('qtd_itens_pedido')
    .size()
    .reset_index(name='qtd_pedidos')
    .sort_values('qtd_itens_pedido')
)

dist_itens

,qtd_itens_pedido,qtd_pedidos
0,1,88863
1,2,7516
2,3,1322
3,4,505
4,5,204
5,6,198
6,7,22
7,8,8
8,9,3
9,10,8


## Junção da base agregada com informações do pedido

Após a agregação dos dados de `order_items`, passamos a ter uma tabela onde cada linha representa um pedido, contendo informações econômicas consolidadas, como quantidade de itens e valores financeiros.

No entanto, para que essa tabela represente de forma completa o conceito de **pedido**, é necessário enriquecê-la com informações operacionais e temporais provenientes do dataset `orders`.

Essas informações descrevem o **ciclo de vida do pedido**, incluindo seu status e os principais marcos temporais desde a compra até a entrega.

Neste bloco, realizaremos:
- A seleção consciente das colunas relevantes do dataset `orders`
- A junção dos dados utilizando `order_id` como chave
- A preservação do grão da análise (1 linha = 1 pedido)
- Validações simples para garantir consistência após o merge

O objetivo desta etapa **não é criar métricas finais**, mas sim **modelar uma tabela de pedidos enriquecida**, que servirá como base confiável para a construção de métricas no próximo notebook.

In [140]:
orders_cols = [
    'order_id',
    'order_status',
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]

orders_enriquecido = orders_df[orders_cols]

pedido_modelado = (
    pedido_agregado
    .merge(
        orders_enriquecido,
        on='order_id',
        how='left'
    )
)

pedido_modelado.head()

,order_id,qtd_itens_pedido,valor_total_itens,valor_total_frete,valor_total_pedido,ticket_medio_item,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,00010242fe8c5a6d1ba2dd792cb16214,1,58.90,13.29,72.19,58.90,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00
1,00018f77f2f0320c557190d7a144bdd3,1,239.90,19.93,259.83,239.90,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00
2,000229ec398224ef6ca0657da4fc703e,1,199.00,17.87,216.87,199.00,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00
3,00024acbcdf0a6daa1e931b038114c75,1,12.99,12.79,25.78,12.99,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,199.90,18.14,218.04,199.90,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00


## Validações pós-junção

Após a junção entre a base agregada de pedidos (`pedido_agregado`) e o dataset de pedidos (`orders`), é fundamental realizar validações básicas para garantir que a modelagem preservou corretamente a estrutura esperada.

Nesta etapa, realizaremos verificações simples para confirmar que:

- O número de linhas permaneceu inalterado após o `merge`
- Cada linha continua representando um único pedido
- Não houve duplicação de `order_id`
- A junção não introduziu inconsistências evidentes

Essas validações não têm como objetivo corrigir dados, mas sim **confirmar a integridade estrutural da tabela modelada**, assegurando que ela está pronta para análises mais profundas nos próximos notebooks.

In [141]:
print("Linhas em pedido_agregado:", pedido_agregado.shape[0])
print("Linhas em pedido_modelado:", pedido_modelado.shape[0])

Linhas em pedido_agregado: 98666
Linhas em pedido_modelado: 98666


In [142]:
pedido_modelado['order_id'].is_unique

True

In [143]:
pedido_modelado.isna().sum()

order_id                            0
qtd_itens_pedido                    0
valor_total_itens                   0
valor_total_frete                   0
valor_total_pedido                  0
ticket_medio_item                   0
order_status                        0
order_purchase_timestamp            0
order_approved_at                  14
order_delivered_carrier_date     1009
order_delivered_customer_date    2190
order_estimated_delivery_date       0
dtype: int64

In [144]:
pedido_modelado.head()

,order_id,qtd_itens_pedido,valor_total_itens,valor_total_frete,valor_total_pedido,ticket_medio_item,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,00010242fe8c5a6d1ba2dd792cb16214,1,58.90,13.29,72.19,58.90,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00
1,00018f77f2f0320c557190d7a144bdd3,1,239.90,19.93,259.83,239.90,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00
2,000229ec398224ef6ca0657da4fc703e,1,199.00,17.87,216.87,199.00,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00
3,00024acbcdf0a6daa1e931b038114c75,1,12.99,12.79,25.78,12.99,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,199.90,18.14,218.04,199.90,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00


## Distribuição das métricas do pedido

Com a tabela de pedidos modelada e validada, o próximo passo é analisar a distribuição das principais métricas numéricas construídas até aqui.

O objetivo desta etapa é compreender:

- Como os valores se distribuem entre os pedidos
- Se existem concentrações, caudas longas ou assimetrias
- Se a maioria dos pedidos é composta por poucos itens e valores baixos
- Se há pedidos que fogem significativamente do padrão geral

Nesta fase, **não buscamos conclusões de negócio**, mas sim **entendimento estatístico e estrutural** das métricas, que servirá de base para interpretações futuras no notebook de métricas.

In [145]:
pedido_modelado[
    ['qtd_itens_pedido', 'valor_total_pedido', 'valor_total_frete']
].describe()

,qtd_itens_pedido,valor_total_pedido,valor_total_frete
count,98666.000000,98666.000000,98666.000000
mean,1.141731,160.577638,22.823562
std,0.538452,220.466087,21.650909
min,1.000000,9.590000,0.000000
25%,1.000000,61.980000,13.850000
50%,1.000000,105.290000,17.170000
75%,1.000000,176.870000,24.040000
max,21.000000,13664.080000,1794.960000


In [146]:
pedido_modelado['qtd_itens_pedido'].value_counts().sort_index()

qtd_itens_pedido
1     88863
2      7516
3      1322
4       505
5       204
6       198
7        22
8         8
9         3
10        8
11        4
12        5
13        1
14        2
15        2
20        2
21        1
Name: count, dtype: int64

In [147]:
(
    pedido_modelado['qtd_itens_pedido']
    .value_counts(normalize=True)
    .sort_index()
)

qtd_itens_pedido
1     0.900645
2     0.076176
3     0.013399
4     0.005118
5     0.002068
6     0.002007
7     0.000223
8     0.000081
9     0.000030
10    0.000081
11    0.000041
12    0.000051
13    0.000010
14    0.000020
15    0.000020
20    0.000020
21    0.000010
Name: proportion, dtype: float64

In [148]:
pedido_modelado[['valor_total_pedido', 'valor_total_frete']].quantile(
    [0.25, 0.5, 0.75, 0.9, 0.95]
)

,valor_total_pedido,valor_total_frete
0.25,61.980,13.850
0.50,105.290,17.170
0.75,176.870,24.040
0.90,307.685,39.495
0.95,450.530,54.960


## Conversão consciente de colunas temporais

Antes da criação de métricas temporais derivadas, é necessário garantir que todas as colunas de data estejam corretamente tipadas como `datetime`.

Durante o processo de junção entre pedidos e itens, as colunas temporais são herdadas do dataset `orders`, porém permanecem como `object` caso não sejam convertidas explicitamente.

Nesta etapa realizamos:
- A conversão explícita das colunas temporais relevantes
- Uma validação simples dos tipos após a conversão

Este passo é fundamental para garantir a consistência e validade das métricas temporais derivadas.

In [149]:
colunas_temporais = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]

for col in colunas_temporais:
    if col in pedido_modelado.columns:
        pedido_modelado[col] = pd.to_datetime(pedido_modelado[col])

In [150]:
pedido_modelado[colunas_temporais].dtypes

order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object

## Criação de métricas temporais derivadas

Além das métricas quantitativas relacionadas a valor e quantidade de itens, um pedido também pode ser analisado a partir de sua dimensão temporal.

Nesta etapa, criamos métricas temporais derivadas a partir das datas disponíveis no dataset de pedidos. Essas métricas permitem compreender:

- O tempo necessário para envio do pedido
- O tempo total até a entrega ao cliente
- Possíveis atrasos ou variações no fluxo logístico

Essas métricas **não representam KPIs finais**, mas sim **atributos estruturais do pedido**, fundamentais para análises posteriores de desempenho operacional e experiência do cliente.

O foco aqui é apenas **construir e validar as métricas**, sem ainda interpretá-las sob uma ótica de negócio.

In [151]:
# Tempo entre compra e aprovação (em horas)
pedido_modelado['tempo_aprovacao_horas'] = (
    pedido_modelado['order_approved_at'] 
    - pedido_modelado['order_purchase_timestamp']
).dt.total_seconds() / 3600

In [152]:

# Tempo entre aprovação e envio ao transportador (em dias)
pedido_modelado['tempo_envio_dias'] = (
    pedido_modelado['order_delivered_carrier_date'] 
    - pedido_modelado['order_approved_at']
).dt.days

In [153]:
# Tempo total entre compra e entrega ao cliente (em dias)
pedido_modelado['tempo_entrega_dias'] = (
    pedido_modelado['order_delivered_customer_date'] 
    - pedido_modelado['order_purchase_timestamp']
).dt.days

In [154]:
# Diferença entre data real de entrega e data estimada (em dias)
pedido_modelado['diferenca_entrega_estimada_dias'] = (
    pedido_modelado['order_delivered_customer_date'] 
    - pedido_modelado['order_estimated_delivery_date']
).dt.days

In [157]:
pedido_modelado[
    [
        'tempo_aprovacao_horas',
        'tempo_envio_dias',
        'tempo_entrega_dias',
        'diferenca_entrega_estimada_dias'
    ]
].describe()

,tempo_aprovacao_horas,tempo_envio_dias,tempo_entrega_dias,diferenca_entrega_estimada_dias
count,98652.000000,97643.000000,96476.000000,96476.000000
mean,10.321315,2.301455,12.094086,-11.876881
std,21.478679,3.559117,9.551746,10.183854
min,0.000000,-172.000000,0.000000,-147.000000
25%,0.215000,0.000000,6.000000,-17.000000
50%,0.343333,1.000000,10.000000,-12.000000
75%,14.529653,3.000000,15.000000,-7.000000
max,1450.866389,125.000000,209.000000,188.000000


## Encerramento do notebook

Neste notebook realizamos a **modelagem da base de pedidos**, integrando informações de itens, valores financeiros e datas operacionais em uma única tabela estruturada no nível de pedido.

Ao longo do processo:
- Agregamos os dados de `order_items` no grão correto (1 linha = 1 pedido)
- Enriquecemos a base com informações do ciclo de vida do pedido a partir de `orders`
- Garantimos a tipagem correta das colunas temporais
- Construímos métricas quantitativas e temporais derivadas

O resultado é uma tabela de pedidos **consistente, validada e pronta para análise**, que servirá como base para a exploração e interpretação das métricas no próximo notebook.

No Notebook 04, o foco será **analisar e interpretar essas métricas**, identificando padrões, assimetrias e possíveis insights a partir dos dados modelados.